In [1]:
from io import StringIO

import numpy as np
import pandas as pd
import polars as pl

# To predict number of proteins, use species numbers from Larsen 2017

## Projected number of species from Larsen 2017

Using data from Table 1, Scenario 1 (Parasites with intermediate parasite richness) from:

    Larsen, B. B., Miller, E. C., Rhodes, M. K. & Wiens, J. J. Inordinate Fondness Multiplied and Redistributed: the Number of Species on Earth and the New Pie of Life. Q. Rev. Biol. 92, 229–265 (2017).


- Animals includes cryptic species for arthropods and 1 apicomplexans (e.g. Malaria parasite) per arthropod


In [29]:
250e6 / 500e3

500.0

In [30]:
(500 * 48) / 60 / 60

6.666666666666667

##### 

In [31]:
larsen2017 = pd.Series(
    {
        "Animals": 163.2e6,
        "Plants": 0.340e6,
        "Fungi": 165.6e6,
        "Protists": 163.2e6,
        "Bacteria": 1.746e9,
    },
    name="n_species",
)

larsen2017.sum()

np.float64(2238340000.0)

## Let's say that 1% of "Bacteria" are Archea because that's the current distribution in NCBI genomes

https://www.ncbi.nlm.nih.gov/datasets/genome/

- 2.54M Bacterial genomes
- 29.05k Archeal genomes

In [32]:
29.05e3 / (2.54e6 + 29.05e3)

0.011307681827913042

In [33]:
larsen2017_archea_bacteria = larsen2017.copy()
larsen2017_archea_bacteria["Archaea"] = 0.01 * larsen2017["Bacteria"]
larsen2017_archea_bacteria["Bacteria"] = 0.99 * larsen2017["Bacteria"]
larsen2017_archea_bacteria

Animals     1.632000e+08
Plants      3.400000e+05
Fungi       1.656000e+08
Protists    1.632000e+08
Bacteria    1.728540e+09
Archaea     1.746000e+07
Name: n_species, dtype: float64

In [34]:
larsen2017_domain_to_kingdom = pd.DataFrame(
    {
        "domain": [
            "Eukaryota",
            "Eukaryota",
            "Eukaryota",
            "Eukaryota",
            "Bacteria",
            "Archaea",
        ],
        "kingdom": ["Animals", "Plants", "Fungi", "Protists", "Bacteria", "Archaea"],
    }
)
larsen2017_domain_to_kingdom = larsen2017_domain_to_kingdom.join(
    larsen2017_archea_bacteria, on="kingdom"
)
larsen2017_domain_to_kingdom

,domain,kingdom,n_species
0,Eukaryota,Animals,1.632000e+08
1,Eukaryota,Plants,3.400000e+05
2,Eukaryota,Fungi,1.656000e+08
3,Eukaryota,Protists,1.632000e+08
4,Bacteria,Bacteria,1.728540e+09
5,Archaea,Archaea,1.746000e+07


## Add number of genes per group

In [35]:
# Prediction of number of proteins per gene
n_proteins_multiplier_larsen2017 = pd.Series(
    {
        # "Typical" bacterial genome is 5000 genes: https://pmc.ncbi.nlm.nih.gov/articles/PMC4361730/
        "Bacteria": 5000,
        # "Typical" bacterial genome is 5000 genes: https://pmc.ncbi.nlm.nih.gov/articles/PMC4361730/
        "Archaea": 5000,
        # Animal predicted ~20,000 from personal intiution: Humans have 20k, Mouse have ~15k, and bivalves like oysters have 30k
        # Also, this 2013 paper: https://pmc.ncbi.nlm.nih.gov/articles/PMC3737309/
        # Decreased from 20k -> 15k since Larsen 2017 predicts majoriy of animal species to be arthropods, which have 10-20k genes generally
        "Animals": 15000,
        # Fungal predicted ~ 11,000 genes/genome from https://pmc.ncbi.nlm.nih.gov/articles/PMC6078396/
        "Fungi": 11000,
        # Plant average of 32,000 genes/genome from https://academic.oup.com/bfg/article/13/4/308/2845968?login=false
        "Plants": 32000,
        # https://ngdc.cncb.ac.cn/p10k/browse/genome
        # Protist 10,000 Genomes Project. The Innovation, 2020, 1(3). (PMID: 34557722)
        # The P10K Database: A Data Portal for the Protist 10,000 Genomes Project (In Preparation)
        # Looked at the high-quality anotations from here and made a guess
        "Protists": 7500,
    },
    name="n_genes",
)
n_proteins_multiplier_larsen2017

Bacteria     5000
Archaea      5000
Animals     15000
Fungi       11000
Plants      32000
Protists     7500
Name: n_genes, dtype: int64

In [36]:
larsen2017_domain_to_kingdom_with_genes = larsen2017_domain_to_kingdom.join(
    n_proteins_multiplier_larsen2017, on="kingdom"
)
larsen2017_domain_to_kingdom_with_genes

,domain,kingdom,n_species,n_genes
0,Eukaryota,Animals,1.632000e+08,15000
1,Eukaryota,Plants,3.400000e+05,32000
2,Eukaryota,Fungi,1.656000e+08,11000
3,Eukaryota,Protists,1.632000e+08,7500
4,Bacteria,Bacteria,1.728540e+09,5000
5,Archaea,Archaea,1.746000e+07,5000


## Lets do per 100k of species to reduce the compute complexity

Divide by 100k, then take the ceiling (round up), and convert to integer so we can make the circle visualization

In [37]:
larsen2017_domain_to_kingdom_with_genes["n_ten_thousand_species"] = np.ceil(
    larsen2017_domain_to_kingdom_with_genes.n_species / 1e4
).astype(int)
larsen2017_domain_to_kingdom_with_genes["n_genes_per_ten_thousand"] = (
    larsen2017_domain_to_kingdom_with_genes.n_genes * 1e4
).astype(int)


larsen2017_domain_to_kingdom_with_genes

,domain,kingdom,n_species,n_genes,n_ten_thousand_species,n_genes_per_ten_thousand
0,Eukaryota,Animals,1.632000e+08,15000,16320,150000000
1,Eukaryota,Plants,3.400000e+05,32000,34,320000000
2,Eukaryota,Fungi,1.656000e+08,11000,16560,110000000
3,Eukaryota,Protists,1.632000e+08,7500,16320,75000000
4,Bacteria,Bacteria,1.728540e+09,5000,172854,50000000
5,Archaea,Archaea,1.746000e+07,5000,1746,50000000


## Now lets create the table where each row is one of 1 million species

In [38]:
dfs = []


def create_species_per_row_table(df, n_species_col="n_ten_thousand_species"):
    this_domain_kingdom = pl.DataFrame(df)
    kingdom = df.kingdom.values[0]

    n_species = df[n_species_col].values[0]

    print(f"Domain: {domain}, Kingdom: {kingdom}, {n_species_col}: {n_species:,}")
    df_predicted_n_proteins = (
        this_domain_kingdom.select(
            pl.all().repeat_by(int(n_species)).flatten()  # .cast(pl.UInt64)
        )
        .with_row_index()
        .with_columns(pl.col(n_species_col).cast(pl.UInt64))
        .with_columns(
            pl.col("index")
            .cast(pl.String)
            .str.zfill(10)
            .str.replace("^", f"10k_{kingdom}_")
            .alias("organism_number")
            # pl.concat_str([kingdom, pl.col("index")]).alias("organism_number")
        )
    )
    return df_predicted_n_proteins


for (domain, kingdom), df in larsen2017_domain_to_kingdom_with_genes.groupby(
    ["domain", "kingdom"]
):
    this_domain_kingdom = pl.DataFrame(df)
    kingdom = df.kingdom.values[0]

    n_species = df.n_ten_thousand_species.values[0]

    print(f"Domain: {domain}, Kingdom: {kingdom}, Thousands of species: {n_species:,}")
    df_predicted_n_proteins = (
        this_domain_kingdom.select(
            pl.all().repeat_by(int(n_species)).flatten()  # .cast(pl.UInt64)
        )
        .with_row_index()
        .with_columns(pl.col("n_ten_thousand_species").cast(pl.UInt64))
        .with_columns(
            pl.col("index")
            .cast(pl.String)
            .str.zfill(10)
            .str.replace("^", f"10k_{kingdom}_")
            .alias("organism_number")
            # pl.concat_str([kingdom, pl.col("index")]).alias("organism_number")
        )
    )
    dfs.append(df_predicted_n_proteins)
larsen2017_predicted_n_proteins = pl.concat(dfs)
larsen2017_predicted_n_proteins

Domain: Archaea, Kingdom: Archaea, Thousands of species: 1,746
Domain: Bacteria, Kingdom: Bacteria, Thousands of species: 172,854
Domain: Eukaryota, Kingdom: Animals, Thousands of species: 16,320
Domain: Eukaryota, Kingdom: Fungi, Thousands of species: 16,560
Domain: Eukaryota, Kingdom: Plants, Thousands of species: 34
Domain: Eukaryota, Kingdom: Protists, Thousands of species: 16,320


index,domain,kingdom,n_species,n_genes,n_ten_thousand_species,n_genes_per_ten_thousand,organism_number
u32,str,str,f64,i64,u64,i64,str
0,"""Archaea""","""Archaea""",1.746e7,5000,1746,50000000,"""10k_Archaea_0000000000"""
1,"""Archaea""","""Archaea""",1.746e7,5000,1746,50000000,"""10k_Archaea_0000000001"""
2,"""Archaea""","""Archaea""",1.746e7,5000,1746,50000000,"""10k_Archaea_0000000002"""
3,"""Archaea""","""Archaea""",1.746e7,5000,1746,50000000,"""10k_Archaea_0000000003"""
4,"""Archaea""","""Archaea""",1.746e7,5000,1746,50000000,"""10k_Archaea_0000000004"""
…,…,…,…,…,…,…,…
16315,"""Eukaryota""","""Protists""",1.632e8,7500,16320,75000000,"""10k_Protists_0000016315"""
16316,"""Eukaryota""","""Protists""",1.632e8,7500,16320,75000000,"""10k_Protists_0000016316"""
16317,"""Eukaryota""","""Protists""",1.632e8,7500,16320,75000000,"""10k_Protists_0000016317"""


In [39]:
larsen2017_predicted_n_proteins.columns

['index',
 'domain',
 'kingdom',
 'n_species',
 'n_genes',
 'n_ten_thousand_species',
 'n_genes_per_ten_thousand',
 'organism_number']

In [40]:
f"{larsen2017_predicted_n_proteins['n_genes_per_ten_thousand'].sum():,}"

'14,234,480,000,000'

In [41]:
larsen2017_predicted_n_proteins
larsen2017_predicted_n_proteins.write_parquet(
    "../data/predicted_n_proteins_larsen2017.parquet"
)